In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from collections import defaultdict
from joblib import Parallel, delayed

import sqlite3
import sys
import time
import math
import tqdm
import datetime
import os
import pickle
from pathlib import Path
import multiprocessing

from glicko2 import Player

if os.path.exists('/workspace/data'):
    # Load the dictionary of DataFrames from the pickle
    data_path = '/workspace/data/'
else:
    data_path = '../data/'

## Loading SQLite Database into Pandas DataFrames

The following code connects to an SQLite database (`melee_player_database.db`) and converts each table within the database into a pandas DataFrame. The DataFrames will be stored in a dictionary, where each key corresponds to the table name with `_df` appended, and the values are the respective DataFrames.

### Steps:

1. **Database Connection**: We use the `sqlite3` library to connect to the SQLite database file.
2. **Retrieve Table Names**: A query retrieves all the table names in the database.
3. **Convert Tables to DataFrames**: For each table:
   - The table is loaded into a pandas DataFrame using `pd.read_sql()`.
   - We check each column to see if any data is JSON-formatted (lists or dictionaries). If so, we convert these columns from strings into their corresponding Python objects using `json.loads()`.
4. **Store DataFrames**: The DataFrames are stored in a dictionary, where the key is the table name with a `_df` suffix, and the value is the DataFrame.
5. **Database Connection Closed**: Once all tables are loaded into DataFrames, the database connection is closed.

### Example:
If the database contains a table named `players`, the corresponding DataFrame will be stored in the dictionary with the key `players_df`, and can be accessed as:

```python
players_df = dfs['players_df']


In [2]:
# Function to get the table names
def get_table_names(conn):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return pd.read_sql(query, conn)['name'].tolist()

# Function to load tables into DataFrames
def load_tables_to_dfs(conn):
    table_names = get_table_names(conn)
    dataframes = {}
    
    for table in table_names:
        # Load table into a DataFrame
        df = pd.read_sql(f"SELECT * FROM {table}", conn)
        
        # Detect and convert JSON formatted columns (if any)
        for col in df.columns:
            # Check if any entry in the column is a valid JSON (list or dictionary)
            if df[col].apply(lambda x: isinstance(x, str)).all():
                try:
                    # Try parsing the column as JSON
                    df[col] = df[col].apply(lambda x: json.loads(x) if pd.notnull(x) else x)
                except (json.JSONDecodeError, TypeError):
                    # If it fails, skip the column
                    pass
        
        # Store the DataFrame with table name + '_df'
        dataframes[f"{table}_df"] = df
        
    return dataframes

if os.path.exists(data_path + 'dfs_dict.pkl'):
    cell_has_run = True
    # Load the dictionary of DataFrames from the pickle
    with open(data_path + 'dfs_dict.pkl', 'rb') as f:
        dfs = pickle.load(f)
# Check if the flag variable exists in the global scope so that this code does not run twice
if 'cell_has_run' not in globals():
    path = data_path + "melee_player_database.db"
    
    # Connect to the database
    conn = sqlite3.connect(path)

    # Convert each table into a DataFrame
    dfs = load_tables_to_dfs(conn)

    # Close the connection
    conn.close()

    # Now, you have a dictionary 'dfs' where each key is the table name with '_df' suffix and value is the corresponding DataFrame.
    # For example, to access the DataFrame for a table called 'players':
    # players_df = dfs['players_df']

    dfs['tournament_info_df']['start'] = pd.to_datetime(dfs['tournament_info_df']['start'], unit='s')
    dfs['tournament_info_df']['end'] = pd.to_datetime(dfs['tournament_info_df']['end'], unit='s')

    
    # Set the flag to indicate that the cell has been run
    cell_has_run = True

### Here we adjust the data types of the dataframes so that they are the correct type. (This will be updated as needed.)

In [3]:
dfs['sets_df']['best_of'] = dfs['sets_df']['best_of'].fillna(0).astype(int) 

### Here we make dataframes that we will use and print the head.

The integers in 'characters' count the number of games the player has played that character. (We verify this for Zain below.)

In [ ]:
players_df = dfs['players_df']
players_df.head()

In [ ]:
ranking_df = dfs['ranking_df']
ranking_df.head()

In [ ]:
ranking_seasons_df = dfs['ranking_seasons_df']
ranking_seasons_df.head()

In [ ]:
sets_df = dfs['sets_df']
print(f"{sets_df[sets_df['game_data'].apply(lambda x: len(x) > 0)].shape[0] / sets_df.shape[0]:0.01%} percent of sets have some game data)")

sets_df.head()

In [ ]:
tournament_info_df = dfs['tournament_info_df']
tournament_info_df.head()

In [9]:
# Code optimization by Dan
# Basically we want to replace this line in process_tournament with something more efficient:
#
#      tournament_sets_df = sets_df[sets_df['tournament_key'] == tournament_key]
#
# Instead, we can
# - Merge the tournament date info into ``sets_df``
# - Sort by date
# - Store the start/end positions of each tournament in a separate dictionary
# - Use tournament_sets_df = sets_df.iloc[start:end+1] instead.

sets_df = sets_df.merge(tournament_info_df[['key', 'start', 'end']], left_on='tournament_key', right_on='key', how='left')
sets_df = sets_df.drop(labels=['key_y'], axis='columns')
sets_df = sets_df.rename(columns={"key_x": "key"})
sets_df = sets_df.sort_values(by=['end', 'tournament_key']) # Just in case there are tournaments with the exact same end date

In [10]:
# A bit of data cleanup
# TODO: Rerun!
min_date = datetime.datetime(2015, 1, 1)
max_date = datetime.datetime(2024, 12, 31)

sets_df = sets_df[(sets_df['start'] >= min_date) & (sets_df['end'] >= min_date) & (sets_df['start'] <= max_date) & (sets_df['end'] <= max_date)]

In [ ]:
# Example of game data. List of dictionaries.
sets_df[sets_df['game_data'].apply(lambda x: x != [])].iloc[0]['game_data']

## Sets to games

Here, we start extracting the individual games (each as one row) from sets.

Because it would take an ABSURDLY long time to compute the ELOs of (specific_player)/(specific_character) \union (anyone)/(specific_opponent_character), we will instead approximate as follows:
* compute the ELOs of (anyone)/(specific_opponent_character) (ex: fox vs fox, to guage vaguely how good each fox player is)
* Having those values fixed, compute the ELO of (specific_player)/(specific_character) (ex: aMSa/yoshi vs all fixed fox player ELOs) in their matches against any such player from before.

In [12]:
# Assumes dataset_generation/game_data_extractor.ipynb was run
game_data_df = pd.read_pickle(data_path + 'individual_game_data.pkl')

In [ ]:
all_characters = list(set(list(game_data_df['p1_char'].unique()) + list(game_data_df['p1_char'].unique())))
all_characters

## First, compute rankings for character vs same character

In [ ]:
# Dataframes containing only character vs same character

char_vs_char_dataframes = {}

for character in tqdm.tqdm(all_characters, total=len(all_characters)):
    char_vs_char_df = game_data_df[(game_data_df['p1_char'] == character) & (game_data_df['p2_char'] == character)]
    char_vs_char_df = char_vs_char_df.reset_index(drop=True) # Necessary for proper position calculations later

    # To avoid absurd pollution of all the datasets,
    # let's only consider players who have actually played more than a few games
    # against another person with the same character
    MIN_GAMES = 3

    game_players_series = pd.concat([char_vs_char_df['p1_id'], char_vs_char_df['p2_id']])
    total_games = game_players_series.value_counts()
    regular_players = total_games[total_games >= MIN_GAMES]

    char_vs_char_dataframes[character] = char_vs_char_df[(char_vs_char_df['p1_id'].apply(lambda x: x in regular_players.index)) &
                                                         (char_vs_char_df['p2_id'].apply(lambda x: x in regular_players.index))]

char_vs_char_dataframes['mrgameandwatch']

In [ ]:
# Dictionary of dictionaries. First key is the specific character used for the matchup.
# For each one, we will store the start/end indexes of each tournament in each dataframe.
# This will be used for far more efficient processing later
date_positions_by_char = {}

for character in tqdm.tqdm(all_characters, total=len(all_characters)):
    char_games_df = char_vs_char_dataframes[character]
    positions = {}

    initial_date = datetime.datetime(2015, 1, 1)
    interval = datetime.timedelta(weeks=1)

    end_date = char_games_df['end'].max()

    date = initial_date

    # Loop through and test for containment in the interval [date, date+interval)
    while date + interval <= end_date:
        is_in_interval = (sets_df['end'] >= date) & (sets_df['end'] < date + interval)
        is_in_interval = is_in_interval.reset_index(drop=True) # Make absolutely sure it can be used for iloc

        if (~is_in_interval).all(): # all False
            positions[date + interval] = (0,-1)
        else: # At least one True
            positions[date + interval] = (is_in_interval.idxmax(), is_in_interval[::-1].idxmax())

        date += interval

    date_positions_by_char[character] = positions

In [18]:
# TLDR the ELO computation occasionally seems to go negative, and I think this can cause a lot of problems.
# Likewise, the RD value seems to get unusually large occasionally
MIN_ELO = 500.0
MAX_RD = 350.0

def process_tournament(player_ratings_df, player_rds_df, games_df, positions, specific_date, key_is_date, glicko_objects):
    # Get the games for this specific period, using precomputed indices for speed.
    period_df = games_df.iloc[positions[specific_date][0]:positions[specific_date][1]+1]
    unique_players = list(player_ratings_df.columns)

    # Prepare player matches DataFrame.
    # Essentially, one copy where p1 is the player of consideration,
    # and then another copy where p2 is under consideration.
    df_p1 = period_df[['p1_id', 'p2_id', 'winner_id']].copy()
    df_p1.rename(columns={'p1_id': 'player_id', 'p2_id': 'opponent_id'}, inplace=True)
    df_p1['outcome'] = (df_p1['winner_id'] == df_p1['player_id']).astype(int)
    df_p1 = df_p1[['player_id', 'opponent_id', 'outcome']]
    
    df_p2 = period_df[['p2_id', 'p1_id', 'winner_id']].copy()
    df_p2.rename(columns={'p2_id': 'player_id', 'p1_id': 'opponent_id'}, inplace=True)
    df_p2['outcome'] = (df_p2['winner_id'] == df_p2['player_id']).astype(int)
    df_p2 = df_p2[['player_id', 'opponent_id', 'outcome']]

    outcomes_df = pd.concat([df_p1, df_p2], ignore_index=True)

    # Add opponent ELO and RD values
    outcomes_df['opponent_rating'] = outcomes_df['opponent_id'].apply(lambda x: glicko_objects[x].getRating())
    outcomes_df['opponent_rd'] = outcomes_df['opponent_id'].apply(lambda x: glicko_objects[x].getRd())

    # Compile the opponent data into lists (one for ratings, one for rds, one for outcomes) for every player
    grouped = outcomes_df.groupby('player_id').agg({
        'opponent_rating': list,
        'opponent_rd': list,
        'outcome': list
    }).reset_index()

    # We will need to update the glicko objects separately for players that did or didn't play in this rating period
    players_with_games = list(grouped['player_id'])
    players_without_games = [x for x in unique_players if x not in players_with_games]

    #First, let's handle the players that did play games in this period
    def update_glicko(row):
        glicko_objects[row['player_id']].update_player(row['opponent_rating'], row['opponent_rd'], row['outcome'])

    grouped.apply(update_glicko, axis=1)

    # Now let's handle players that did NOT play games in this period
    for player in players_without_games:
        glicko_objects[player].did_not_compete()

    # This might be a bug in the glicko2 library that we are using,
    # but occasionally we get weird values for the rating and/or RD
    # and this DOES very occasionally seem to cause the whole thing to crash
    # (division by zero somewhere)
    # and so we use professional-grade duct tape here
    for player in unique_players:
        if glicko_objects[player].getRating() < MIN_ELO:
            glicko_objects[player].setRating(MIN_ELO)

        if glicko_objects[player].getRd() > MAX_RD:
            glicko_objects[player].setRd(MAX_RD)

    # Now update that specific row of the player ratings dataframe with the new glicko data
    date = specific_date if key_is_date else games_df.iloc[positions[specific_date][0]]['end']

    #new_player_data = {}
    #for player in unique_players:
    #    new_player_data[player] = glicko_objects[player].getRating()

    # These should be in the same order as the columns, as unique_players was used to create the columns as well.
    # This is slightly more efficient than using a dictionary.
    new_player_ratings = [glicko_objects[player].getRating() for player in unique_players]
    new_player_rds     = [glicko_objects[player].getRd()     for player in unique_players]

    player_ratings_df.loc[date] = new_player_ratings
    player_rds_df.loc[date]     = new_player_rds

In [ ]:
# Basically, we will instead create a dataframe with index of dates, columns of players
player_ratings_char_vs_char = {}
player_rds_char_vs_char = {}

for character in tqdm.tqdm(all_characters, total=len(all_characters)):
    games_df = char_vs_char_dataframes[character]
    positions = date_positions_by_char[character]

    unique_players = list(set(list(games_df['p1_id'].unique()) + list(games_df['p2_id'].unique())))

    # Note that this technically starts at initial_date + interval, not initial_date
    dates = [x for x in positions]

    # Convenient store of glicko objects
    glicko_objects = {}
    for player in unique_players:
        glicko_objects[player] = Player()

    # Initial row of default ELO values, at the earliest possible date.
    # Note that we are also pre-allocating the dataframe with the rest of the possible dates.
    player_ratings_df = pd.DataFrame([[1500.0] * len(unique_players)], columns=unique_players, index=[initial_date] + dates)
    player_rds_df = pd.DataFrame([[350.0] * len(unique_players)], columns=unique_players, index=[initial_date] + dates)

    # Loop over the individual date intervals above.
    for specific_date in dates:
        process_tournament(player_ratings_df, player_rds_df, games_df, positions, specific_date, True, glicko_objects)

    Path(data_path + 'char_vs_char/').mkdir(parents=True, exist_ok=True)
    player_ratings_df.to_pickle(data_path + 'char_vs_char/' + character + '.pkl')
    #print('Size in memory: {0:.2f}'.format(sys.getsizeof(player_ratings_df) / (1024.0*1024.0)))
    
    player_ratings_char_vs_char[character] = player_ratings_df
    player_rds_char_vs_char[character] = player_rds_df

## Compute the rankings for player/char/char

In [ ]:
# Good for testing
# 1021/yoshi - aMSa
# 19554/fox - Cody

# First, we build the set of rounds with one player and one opponent.
# Each original row in game_data_df will contribute twice, with each player swapping the above roles.

# For convenience, we can restrict our attention to players who actually have a reasonable amount of data with a certain character
MIN_GAMES = 1

game_players_series = pd.concat([game_data_df['p1_id'], game_data_df['p2_id']])
total_games = game_players_series.value_counts()
regular_players = total_games[total_games >= MIN_GAMES]

print("{0} regular player/char combos".format(len(regular_players.index)))

# Lots of memory usage. Let's just reduce down to what we need.
rounds_df = game_data_df[['p1_id', 'p2_id', 'winner_id', 'p1_char', 'p2_char', 'winner_char', 'start', 'end']]

# Each row should contribute twice, swapping 'player' and 'opponent'
df_p1 = rounds_df[rounds_df['p1_id'].apply(lambda x: x in regular_players.index)].copy()
df_p1.rename(columns={'p1_id': 'player_id', 'p2_id': 'opponent_id',
                      'p1_char': 'player_char', 'p2_char': 'opponent_char'}, inplace=True)
df_p1['outcome'] = (df_p1['winner_id'] == df_p1['player_id']).astype(int)
df_p1 = df_p1[['player_id', 'opponent_id', 'player_char', 'opponent_char', 'outcome', 'start', 'end']]
    
df_p2 = rounds_df[rounds_df['p2_id'].apply(lambda x: x in regular_players.index)].copy()
df_p2.rename(columns={'p2_id': 'player_id', 'p1_id': 'opponent_id',
                      'p2_char': 'player_char', 'p1_char': 'opponent_char'}, inplace=True)
df_p2['outcome'] = (df_p2['winner_id'] == df_p2['player_id']).astype(int)
df_p2 = df_p2[['player_id', 'opponent_id', 'player_char', 'opponent_char', 'outcome', 'start', 'end']]

rounds_df = pd.concat([df_p1, df_p2], ignore_index=True)

# Save some memory - these are probably huge.
del df_p1
del df_p2

In [ ]:
rounds_df

In [ ]:
# Here, we fetch the opponent elos. This could take a few minutes, hopefully not more than 5

# Returns Rating, RD, and a bool for (actually found = True, default values = False)
def get_opponent_elo_rd(row):
    # See how opponent/char compares against (others)/char for the same char
    player_ratings = player_ratings_char_vs_char[row['opponent_char']]
    player_rds = player_rds_char_vs_char[row['opponent_char']]

    # Not in our main list of players
    if row['opponent_id'] not in player_ratings.columns:
        return (1500.0, 350.0, False)

    # No old enough data
    if player_ratings.index[0] > row['start']:
        return (1500.0, 350.0, False)

    # We can take advantage of the fact that the index of player_ratings is always in regular intervals.
    start_date = player_ratings.index[0]
    interval = player_ratings.index[1] - player_ratings.index[0] # I guess we're assuming at least two entries?

    newest_index = int((row['start'] - start_date) / interval)

    # Might actually be out of bounds on the data we have,
    # i.e. 'start' might be well beyond the dates we have data on.
    # In this case, just use the newest piece of data.
    if newest_index >= len(player_ratings.index):
        newest_index = len(player_ratings.index) - 1

    return (player_ratings.iloc[newest_index][row['opponent_id']], player_rds.iloc[newest_index][row['opponent_id']], True)

rounds_df['result'] = rounds_df.apply(get_opponent_elo_rd, axis=1)

rounds_df['opponent_rating'] = rounds_df['result'].apply(lambda x: x[0])
rounds_df['opponent_rd']     = rounds_df['result'].apply(lambda x: x[1])
rounds_df['opponent_found']  = rounds_df['result'].apply(lambda x: x[2])

rounds_df.drop(columns=['result'], inplace=True)

rounds_df

In [34]:
# Compute weekly intervals to group by, quite easily.
start_date = datetime.datetime(2015,1,1)
interval = player_ratings_char_vs_char['fox'].index[1] - player_ratings_char_vs_char['fox'].index[0]

# "Copy of a slice" nonsense, this should fix it.
rounds_df = rounds_df.copy()

# Round up, as this computes the date that receives this elo update.
rounds_df['end_index'] = rounds_df['end'].apply(lambda x: math.ceil((x - start_date) / interval))

In [ ]:
# Group by player/character (pc_combo), opponent character, week index
rounds_df['pc_combo'] = rounds_df['player_id'] + '/' + rounds_df['player_char']

grouped_df = rounds_df[['pc_combo', 'opponent_char', 'end_index',
                        'opponent_rating', 'opponent_rd', 'outcome']].groupby(['pc_combo', 'opponent_char', 'end_index']).agg({
        'opponent_rating': list,
        'opponent_rd': list,
        'outcome': list
    }).reset_index()

grouped_df['player_char_char'] = grouped_df['pc_combo'] + '/' + grouped_df['opponent_char']
grouped_df.drop(columns=['pc_combo', 'opponent_char'], inplace=True)
grouped_df

In [ ]:
# Actually start computing elos for player/char/char combos.
# TODO: This is REALLY slow. Optimize!

# To deal with inlcude_groups=True being deprecated and disallowed soon,
# let's just create a copy of this column
grouped_df['pcc_duplicate'] = grouped_df['player_char_char']

# Parallelization, cause this be SLOW
hyperthreading = True
n_jobs = multiprocessing.cpu_count() // 2 if hyperthreading else multiprocessing.cpu_count()

# Split into separate dataframes and save in separate files.
# This lets us easily run a multiprocessing script later on them.
unique_pcc_combos = list(grouped_df['player_char_char'].unique())
split_pcc_combos = [] # List of lists to filter by

for i in range(0, n_jobs):
    # First n-1 lists will have this length.
    # Last one will have the remainder.
    # This isn't the most even split, but it gets the job done.
    default_length = len(unique_pcc_combos) // n_jobs

    if i != n_jobs - 1:
        split_pcc_combos += [unique_pcc_combos[i*default_length : (i+1)*default_length]]
    else:
        split_pcc_combos += [unique_pcc_combos[i*default_length : ]]

for i, split in enumerate(split_pcc_combos):
    filter = grouped_df['player_char_char'].isin(split)
    split_grouped_df = grouped_df[filter]
    split_grouped_df.to_pickle(data_path + 'char_vs_char_player_rankings_weekly_alt_temp_' + str(i) + '.pkl')

# grouped_df.groupby('player_char_char').apply(compute_pcc_elo, include_groups=False)

## You should now run the separate multiprocessing scripts